<a href="https://colab.research.google.com/github/gpfer/projektR/blob/main/klasifikacija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!npx degit Jainu-s/urldata/al -f

npx: installed 1 in 1.974s
> destination directory is not empty. Using --force, continuing
> cloned Jainu-s/urldata#master


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import base64
import requests

master = "https://raw.githubusercontent.com/gpfer/projektR/main/dataset/klasifikacija.csv"
req = requests.get(master)
req = req.text
print(req)

id,place,time
1,out,day
2,out,day
3,out,day
4,out,day
5,out,night
6,out,day
7,out,day
8,out,day
9,out,day
10,in,night
11,out,day
12,out,day
13,out,day
14,out,day
15,in,day
16,out,day
17,out,day
18,out,day
19,out,day
20,out,night
21,out,day
22,out,day
23,out,day
24,out,day
25,in,night
26,out,day
27,out,day
28,out,day
29,out,day
30,in,day
31,out,day
32,out,day
33,out,day
34,out,day
35,out,night
36,out,day
37,out,day
38,out,day
39,out,day
40,in,night
41,out,day
42,out,day
43,out,day
44,out,day
45,in,night
46,out,day
47,out,day
48,out,day
49,out,day
50,in,night
51,out,day
52,out,day
53,out,day
54,out,day
55,out,night
56,out,day
57,out,day
58,out,day
59,out,day
60,in,day
61,out,day
62,out,day
63,out,day
64,out,day
65,in,night
66,out,day
67,out,day
68,out,day
69,out,day
70,out,night
71,out,day
72,out,day
73,out,day
74,out,day
75,in,night
76,out,day
77,out,day
78,out,day
79,out,day
80,in,day
81,out,day
82,out,day
83,out,day
84,out,day
85,in,night
86,out,day
87,out,day
88,out,day
89,out,day
90

Keirajmo kategije od podataka: [in-day, in-night, out-day, out-night]

In [ ]:
import numpy as np

data = req.text